<a href="https://colab.research.google.com/github/wross3150/Phys-142-Project/blob/main/Physics_142_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm

In [ ]:
# Constants
MZ = 91.188  # GeV
Gamma_Z = 2.4414  # GeV
alpha = 1 / 132.507
GF = 1.16639e-5  # GeV^-2
sin2_thetaW = 0.222246
GeV_to_pb = 3.894e8  # pb^-1
kappa = np.sqrt(2) * GF * MZ**2 / (4 * np.pi * alpha)

# Function to calculate A0 and A1
def calculate_A0_A1(s_hat):
    Qe = -1
    Ve = -0.5 + 2 * sin2_thetaW
    Ae = -0.5
    Qmu = -1
    Vmu = -0.5 + 2 * sin2_thetaW
    Amu = -0.5
    s = s_hat
    A0 = Qe**2 - 2 * Qe * Vmu * Ve * chi1(s_hat) + (Amu**2 + Vmu**2) * (Ae**2 + Ve**2) * chi2(s_hat)
    A1 = -4 * Qe * Ae * Amu * chi1(s_hat) + 8 * Amu * Vmu * Ae * Ve * chi2(s_hat)
    return A0, A1

# Functions to calculate chi1 and chi2
def chi1(s_hat):
    s = s_hat
    return (kappa * s * (s - MZ**2)) / ((s - MZ**2)**2 + (Gamma_Z * MZ)**2)

def chi2(s_hat):
    s = s_hat
    return kappa**2 * s**2 / ((s - MZ**2)**2 + (Gamma_Z * MZ)**2)

def p(s_hat, cos_theta):
  A0, A1 = calculate_A0_A1(s_hat)
  return alpha**2 / (4 * s_hat) * (A0 * (1 + cos_theta**2) + A1 * cos_theta)

# Define X, Y boundaries

Set $x = E_{CM}$ and $y = cos(\theta)$

In [ ]:
X_MIN = 10
X_MAX = 200
Y_MIN = -1
Y_MAX = 1

X_MIN = -50
X_MAX = 50
Y_MIN = -5
Y_MAX = 5

#X = np.sqrt(s_hat)
#Y = cos_theta

# Lego Plot

In [26]:
def lego_plot(xAmplitudes, yAmplitudes, nBins, xLabel, yLabel, title):
    x = np.array(xAmplitudes)  # turn x,y data into numpy arrays
    y = np.array(yAmplitudes)  # useful for regular matplotlib arrays

    fig = plt.figure()  # create a canvas, tell matplotlib it's 3d
    ax = fig.add_subplot(111, projection="3d")

    # make histograms - set bins
    hist, xedges, yedges = np.histogram2d(x, y, bins=(nBins, nBins))
    xpos, ypos = np.meshgrid(xedges[:-1] + xedges[1:], yedges[:-1] + yedges[1:])

    xpos = xpos.flatten() / 2.0
    ypos = ypos.flatten() / 2.0
    zpos = np.zeros_like(xpos)

    dx = xedges[1] - xedges[0]
    dy = yedges[1] - yedges[0]
    dz = hist.flatten()

    cmap = mpl.cm.jet
    max_height = np.max(dz)  # get range of colorbars so we can normalize
    min_height = np.min(dz)
    # scale each z to [0,1], and get their rgb values
    rgba = [cmap((k - min_height) / max_height) for k in dz]

    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color=rgba, zsort="average")
    plt.title(title)
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.xlim(XMIN, XMAX)
    plt.ylim(YMIN, YMAX)
    plt.show()

# Find Maximum

In [ ]:
def find_max(p, STEPS, X_HIT = (0.05 * 95), Y_HIT = 0.05):
  E_vals = [np.random.rand() * (X_MAX - X_MIN) + X_MIN]
  cos_vals = [np.random.rand() * (Y_MAX - Y_MIN) + Y_MIN]
  prob_vals = [p(E_vals[0] ** 2, cos_vals[0])]
  best = [E_vals[0], cos_vals[0], prob_vals[0]]
  for i in range(STEPS):
    E_diff = np.random.rand() *  X_HIT
    cos_diff = np.random.rand() * Y_HIT
    E_new = E_vals[-1] + E_diff
    cos_new = cos_vals[-1] + cos_diff
    new_prob = p(E_new ** 2, cos_new)
    if new_prob > best[2]:
      E_vals.append(E_new)
      cos_vals.append(cos_new)
      prob_vals.append(new_prob)
      best = [E_new, cos_new, new_prob]
    elif new_prob / prob_vals[-1] > np.random.rand():
      E_vals.append(E_new)
      cos_vals.append(cos_new)
      prob_vals.append(new_prob)
  return best, E_vals, cos_vals, prob_vals

In [ ]:
def f(x,y):
  return -x**2 - y ** 2

best, E_vals, cos_vals, prob_vals = find_max(f, 100000)

In [ ]:
best

[-0.9337818649143586, -1.3246347059772756, -2.5149514151721033]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

n = 100

# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
#for m, zlow, zhigh in [('o', -50, -25), ('^', -30, -5)]:
xs = E_vals
ys = cos_vals
zs = prob_vals
ax.scatter(xs, ys, zs)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

# Problem A

In [27]:
def brute_force(nPoints, seed=None):
    nFunctionEval = 0
    yy1_rej_method = []
    yy2_rej_method = []
    maxWeightEncounteredRej = -1.0e20
    generator = np.random.RandomState(seed=seed)
    while len(yy1_rej_method) < nPoints:
        rr = generator.uniform(size=3)
        yy1, yy2 = XMIN + rr[0] * (XMAX - XMIN), YMIN + rr[1] * (YMAX - YMIN)
        nFunctionEval += 1
        f_val = f(yy1, yy2)
        if f_val > maxWeightEncounteredRej:
            maxWeightEncounteredRej = f_val
        if f_val > F_VAL_MAX:
            print(
                f" f_val={f_val} exceeds F_VAL_MAX={F_VAL_MAX}, program will now exit"
            )
            exit(99)
        if f_val / F_VAL_MAX > rr[2]:
            yy1_rej_method.append(yy1)
            yy2_rej_method.append(yy2)
    return {
        "yy1": yy1_rej_method,
        "yy2": yy2_rej_method,
        "nFunEval": nFunctionEval,
        "maxWeightEncountered": maxWeightEncounteredRej,
    }

# Problem B

## Setup Intervals

In [ ]:
def setup_intervals(NN=100, KK=2000, nIterations=4000, alpha_damp=1.5, seed=None):
    """
    Input:
        NN: Number of intervals in [XMIN, XMAX] or [YMIN, YMAX]
        KK: function evaluations per iteration
        nIterations: number of iterations
        alpha_damp: damping parameter in the Vegas algorithm
    Return:
        Intervals specified by xLow, yLow: each is a 1D numpy array of size NN+1, with
        xLow[0] = 0, xLow[NN] = ym; yLow[0] = 0, yLow[NN] = ym
    """

    # intitial intervals: uniform intervals between XMIN/YMIN and XMAX/YMAX
    xLow = XMIN + (XMAX - XMIN) / NN * np.arange(NN + 1)
    delx = np.ones(NN) * (XMAX - XMIN) / NN
    px = np.ones(NN) / (XMAX - XMIN)  # probability density in each interval
    yLow = YMIN + YMAX / NN * np.arange(NN + 1)
    dely = np.ones(NN) * (YMAX - YMIN) / NN
    py = np.ones(NN) / (YMAX - YMIN)

    generator = np.random.RandomState(seed=seed)
    for _ in range(nIterations):
        ixLow = generator.randint(0, NN, size=KK)
        xx = xLow[ixLow] + delx[ixLow] * generator.uniform(size=KK)
        iyLow = generator.randint(0, NN, size=KK)
        yy = yLow[iyLow] + dely[iyLow] * generator.uniform(size=KK)
        ff = f(xx, yy)
        f2barx = np.array(
            [sum((ff[ixLow == i] / py[iyLow[ixLow == i]]) ** 2) for i in range(NN)]
        )
        fbarx = np.sqrt(f2barx)
        f2bary = np.array(
            [sum((ff[iyLow == i] / px[ixLow[iyLow == i]]) ** 2) for i in range(NN)]
        )
        fbary = np.sqrt(f2bary)
        fbardelxSum = np.sum(fbarx * delx)
        fbardelySum = np.sum(fbary * dely)
        logArgx = fbarx * delx / fbardelxSum
        logArgy = fbary * dely / fbardelySum

        """mmx = KK * pow((logArgx - 1) / np.log(logArgx), alpha_damp)
        mmx = mmx.astype(int)
        mmx = np.where(mmx > 1, mmx, 1)"""
        mmx = KK * logArgx
        mmx = mmx.astype(int)
        mmx = np.where(mmx > 1, mmx, 1)
        """mmy = KK * pow((logArgy - 1) / np.log(logArgy), alpha_damp)
        mmy = mmy.astype(int)
        mmy = np.where(mmy > 1, mmy, 1)"""
        mmy = KK * logArgy
        mmy = mmy.astype(int)
        mmy = np.where(mmy > 1, mmy, 1)

        xLowNew = [xLow[i] + np.arange(mmx[i]) * delx[i] / mmx[i] for i in range(NN)]
        xLowNew = np.concatenate(xLowNew, axis=0)
        yLowNew = [yLow[i] + np.arange(mmy[i]) * dely[i] / mmy[i] for i in range(NN)]
        yLowNew = np.concatenate(yLowNew, axis=0)
        nCombx = int(len(xLowNew) / NN)
        nComby = int(len(yLowNew) / NN)
        i = np.arange(NN)
        xLow[:-1] = xLowNew[i * nCombx]
        yLow[:-1] = yLowNew[i * nComby]
        delx = np.diff(xLow)
        dely = np.diff(yLow)
        px = 1.0 / delx / NN
        py = 1.0 / dely / NN

    return xLow, yLow, delx, dely

## Vegas

In [ ]:
def vegas(
    nPoints,
    vegasRatioFactor,
    NN=100,
    KK=2000,
    nIterations=4000,
    alpha_damp=1.5,
    seed=None,
):
    xLow, yLow, delx, dely = setup_intervals(NN, KK, nIterations, alpha_damp, seed)
    vegasRatioMax = vegasRatioFactor * F_VAL_MAX * NN * NN * delx[NN - 2] * dely[NN - 2]
    nFunctionEval = 0
    yy1_vegas_method = []
    yy2_vegas_method = []
    yy1_vrho_method = []
    yy2_vrho_method = []
    maxWeightEncountered = -1.0e20

    generator = np.random.RandomState(seed=seed)
    while len(yy1_vegas_method) < nPoints:
        ixLow = generator.randint(0, NN)
        xx = xLow[ixLow] + delx[ixLow] * generator.uniform()
        iyLow = generator.randint(0, NN)
        yy = yLow[iyLow] + delx[iyLow] * generator.uniform()
        yy1_vrho_method.append(xx)
        yy2_vrho_method.append(yy)
        nFunctionEval += 1
        f_val = f(xx, yy)
        ratio = f_val * NN * NN * delx[ixLow] * dely[iyLow]
        if ratio > maxWeightEncountered:
            maxWeightEncountered = ratio
        if ratio > vegasRatioMax:
            print(
                f"ratio={ratio} exceeds vegasRatioMax={vegasRatioMax}, yy={yy} program will now exit "
            )
            exit(99)
        if ratio / vegasRatioMax > generator.uniform():
            yy1_vegas_method.append(xx)
            yy2_vegas_method.append(yy)

    return {
        "yy1vrho": yy1_vrho_method,
        "yy2vrho": yy2_vrho_method,
        "yy1vegas": yy1_vegas_method,
        "yy2vegas": yy2_vegas_method,
        "nFunEval": nFunctionEval,
        "maxWeightEncountered": maxWeightEncountered,
        "vegasRatioMax": vegasRatioMax,
    }

# Additional Code